In [1]:
import gym
import numpy as np
from IPython.display import clear_output
import time
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
env = gym.make('ma_gym:PredatorPrey5x5-v0')

In [3]:
len(env.observation_space)

2

In [4]:
env.observation_space[0]

Box([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.], [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.], (28,), float32)

In [5]:
env.prey_pos

{0: None}

In [6]:
env.agent_pos

{0: None, 1: None}

In [7]:
type(env.action_space.sample())

list

# --------------------------------------------------------------------------------------------------------------

## Environment Dynamics:

In [8]:
def state_to_encoding(state):
    encoding = 0
    for i in range(len(state)):
        encoding += (5**(5-i))*int(state[i])
    return encoding

In [9]:
def encoding_to_state(encoding):
    state = ""
    for i in range(5, 0, -1):
        remainder = encoding % 5
        encoding = encoding//5
        state = str(remainder) + state
    remainder = encoding % 5
    state = str(remainder) + state
    return state

In [10]:
def action_to_encoding(action):
    encoding = 0
    for i in range(len(action)):
        encoding += (5**(1-i))*int(action[i])
    return encoding

In [11]:
def encoding_to_action(encoding):
    action = ""
    remainder = encoding % 5
    encoding = encoding//5
    action = str(int(remainder)) + action
    remainder = encoding % 5
    action = str(int(remainder)) + action
    return action

In [71]:
def get_next_state(state, action):
    
    #Encoding in general means the number, state is the string
    state = encoding_to_state(state)
    action = encoding_to_action(action)
    
    good_noop = 0
    noop_1 = 0
    noop_2 = 0
    
    #STATE:
    #First 2, Second 2, Third 2, depict the positions of the 1st pred, 2nd pred, and prey respectively.
    
    #PREDATOR 1:
    
    if action[0] == "0":
        if state[0] != "4":
            state = state[:0] + str(int(state[0]) + 1) + state[1:]
    if action[0] == "1":
        if state[1] != "0":
            state = state[:1] + str(int(state[1]) - 1) + state[2:]
    if action[0] == "2":
        if state[0] != "0":
            state = state[:0] + str(int(state[0]) - 1) + state[1:]
    if action[0] == "3":
        if state[1] != "4":
            state = state[:1] + str(int(state[1]) + 1) + state[2:]
    if action[0] == "4":
        if (abs(int(state[4]) - int(state[0])) == 1 and abs(int(state[5]) - int(state[1])) == 0) or (abs(int(state[4]) - int(state[0])) == 0 and abs(int(state[5]) - int(state[1])) == 1):
            noop_1 = 1
        
    #PREDATOR 2:
    
    if action[1] == "0":
        if state[2] != "4":
            state = state[:2] + str(int(state[2]) + 1) + state[3:]
    if action[1] == "1":
        if state[3] != "0":
            state = state[:3] + str(int(state[3]) - 1) + state[4:]
    if action[1] == "2":
        if state[2] != "0":
            state = state[:2] + str(int(state[2]) - 1) + state[3:]
    if action[1] == "3":
        if state[3] != "4":
            state = state[:3] + str(int(state[3]) + 1) + state[4:]
    if action[1] == "4":
        if (abs(int(state[4]) - int(state[2])) == 1 and abs(int(state[5]) - int(state[3])) == 0) or (abs(int(state[4]) - int(state[2])) == 0 and abs(int(state[5]) - int(state[3])) == 1):
            noop_2 = 1
            
    reward_array = np.zeros((4,))
    state_array = [state, state, state, state]
    prev_state_array = state_array[:]
    
    #All Possible Next States:
    
    if state_array[0][4] != "4":
        state_array[0] = state_array[0][:4] + str(int(state_array[0][4]) + 1) + state_array[0][5:]
    if state_array[1][5] != "0":
        state_array[1] = state_array[1][:5] + str(int(state_array[1][5]) - 1)
    if state_array[2][4] != "0":
        state_array[2] = state_array[2][:4] + str(int(state_array[2][4]) - 1) + state_array[2][5:]
    if state_array[3][5] != "4":
        state_array[3] = state_array[3][:5] + str(int(state_array[3][5]) + 1)
    
            
    #TERMINAL CASES:
    
    for i in range(len(state_array)):

#         if action[0] == "4":
#             if abs(int(state_array[i][4]) - int(state_array[i][0])) == 1 or abs(int(state_array[i][5]) - int(state_array[i][1])) == 1:
#                 noop_1 = 1

#         if action[1] == "4":
#             if abs(int(state_array[i][4]) - int(state_array[i][2])) == 1 or abs(int(state_array[i][5]) - int(state_array[i][3])) == 1:
#                 noop_2 = 1
        
        if (noop_1 + noop_2) == 2:          #Correct Noop
            state_array[i] = state_to_encoding(state_array[i])
            reward_array[i] = 1
        elif (noop_1 + noop_2) == 1:        #Wrong Noop       
            state_array[i] = state_to_encoding(state_array[i])
            reward_array[i] = -0.5
        elif (action[0] == "4" and noop_1 == 0) or (action[1] == "4" and noop_2 == 0):  #Terrible Noop
            state_array[i] = state_to_encoding(state_array[i])
            reward_array[i] = -1
        #Collision:
        elif (state_array[i][:2] == state_array[i][2:4]) or (state_array[i][2:4] == state_array[i][4:]) or (state_array[i][:2] == state_array[i][4:]):
            state_array[i] = state_to_encoding(prev_state_array[:][i])
            reward_array[i] = -0.5
        #Movement Collision:
        elif (state_array[i][:2] == prev_state_array[i][2:4] and prev_state_array[i][:2] == state_array[i][2:4]) or (state_array[i][4:] == prev_state_array[i][2:4] and prev_state_array[i][4:] == state_array[i][2:4]) or (state_array[i][:2] == prev_state_array[i][4:] and prev_state_array[i][:2] == state_array[i][4:]):
            state_array[i] = state_to_encoding(prev_state_array[:][i])
            reward_array[i] = -0.5
        else:
            state_array[i] = state_to_encoding(state_array[i])
            reward_array[i] = -0.01
        
    
    return reward_array, state_array
    

In [15]:
encoded_action = action_to_encoding("31")
encoded_state = state_to_encoding("111224")
reward, next_state = get_next_state(encoded_state, encoded_action)
print(next_state, reward)
encoding_to_state(next_state[1])

[4544, 4538, 4534, 4539] [0. 0. 0. 0.]


'121123'

In [18]:
state = "001144"

In [19]:
#0-->Down
#1-->Left
#2-->Up
#3-->Right
action_dict = {0:"Down", 1:"Left", 2:"Up", 3:"Right", 4:"Noop"}

encoded_state = state_to_encoding(state)
encoded_action = action_to_encoding("20")
state = get_next_state(encoded_state, encoded_action)

## Agent:

In [79]:
class Agent():
    
    def __init__(self, env):
        self.env = env
        self.n = len(self.env.observation_space) + 1              # 1 prey
        self.num_states = 5**(2*self.n)    # 5 rows/columns squared for area and multiplied by number of agents including prey
        self.num_actions = 5*5
        self.values = np.random.rand(self.num_states)            
        self.policy = np.zeros(self.num_states, )
        
        
    def value_iterate(self, theta = 0.01, gamma = 0.5, save = False):
        
        while True:
            delV = 0
            for state in range(self.num_states):
                prev_value = self.values[state]
                action_values = []
                for action in range(self.num_actions):
                    reward_array, next_state_array = get_next_state(state, action)
                    next_state_vals = 0
                    for i in range(len(next_state_array)):
                        if next_state_array[i] == -1:
                            next_state_vals += (0.25*reward_array[i])
                        else:
                            next_state_vals += (0.25*(reward_array[i] + gamma*self.values[next_state_array[i]]))
                    action_values.append(next_state_vals)
                self.values[state] = max(action_values)
                delV = max(delV, abs(prev_value - self.values[state]))
            print(delV)
            if delV < theta:
                #print(action_values)
                break
            
                
        for state in range(self.num_states):
            action_values = []
            for action in range(self.num_actions):
                reward_array, next_state_array = get_next_state(state, action)
                next_state_vals = 0
                for i in range(len(next_state_array)):
                    if next_state_array[i] == -1:
                        next_state_vals += (0.25*reward_array[i])
                    else:
                        next_state_vals += (0.25*(reward_array[i] + gamma*self.values[next_state_array[i]]))
                action_values.append(next_state_vals)
            action_values = np.array(action_values)
            self.policy[state] = np.argmax(action_values)
            #print(action_values, self.policy[state])
        
        if save:
            np.save("DP_policy.npy", self.policy)

                
    def analyse_episode(self):
        
        dones = [False for _ in range(self.env.n_agents)]
        action = [0, 0]
        reward = [0, 0]
        my_reward = [0, 0]
        info = None
        
        _ = self.env.reset()
        count = 0
        
        self.env.render()
        _ = input("Press Enter to start simulation")
        
        while not all(dones):
            
            #clear_output(wait = True)
            self.env.render()
            
            a1_pos = str(self.env.agent_pos[0][0]) + str(self.env.agent_pos[0][1])
            a2_pos = str(self.env.agent_pos[1][0]) + str(self.env.agent_pos[1][1])
            prey_pos = str(self.env.prey_pos[0][0]) + str(self.env.prey_pos[0][1])
            
            state_string = a1_pos + a2_pos + prey_pos
            state_encoding = state_to_encoding(state_string)
            action_encoding = self.policy[state_encoding]
            action_string = encoding_to_action(action_encoding)
            action[0], action[1] = int(action_string[0]), int(action_string[1])
            print(action_dict[action[0]], action_dict[action[1]])
            print("\nCurrent state in their representation = ", state_string)
            my_reward, my_next_state = get_next_state(state_encoding, action_encoding)
            _ = input("Does the action make sense? Press Enter to execute")
            _, reward, dones, info = self.env.step(action)
            print("REWARD = ", my_reward)
            print("Is agent done?: ", dones)
            self.env.render()
            
    def 
            
            
            

In [80]:
agent = Agent(env)

In [81]:
agent.policy = np.load("DP_policy.npy")
print(agent.policy)

[15. 24. 18. ...  6. 24.  7.]


In [74]:
agent.value_iterate(theta = 0.01, gamma = 0.9, save = True)

1.7742722156571444
0.8711045807429753
0.7590532262640346
0.5045372470958434
0.3942077404325697
0.32904660045789647
0.2705332184909153
0.23083617830763026
0.20037164418457776
0.16711729751449167
0.13847796659839595
0.11351792727886023
0.09289153339497025
0.07552232280694815
0.06158720937712037
0.05267876745360223
0.04534007544833196
0.03896131724754692
0.03345883040189701
0.02868752803102481
0.024571720581408663
0.021017932130436456
0.017957460661794045
0.015323904719945602
0.013061453397187783
0.011120191542921987
0.00945691674725957


In [28]:
np.save("DP_policy_gamma001.npy", agent.policy)

In [87]:
agent.env.close()

In [86]:
agent.analyse_episode()

Press Enter to start simulation
Right Right

Current state in their representation =  223133
Does the action make sense? Press Enter to execute
REWARD =  [-0.01 -0.5  -0.5  -0.01]
Is agent done?:  [True, True]
